# Graphs
#### Adjacency list
Given an adjacency list, return:
- the number of nodes in the graph
- the number of edges in the graph
- the degree of a node
- the neighbours of a node.

In [2]:
adjacency_list = [
    [1],       # node 0
    [0,2,5,4], # node 1
    [1,4,5],   # node 2
    [],        # node 3
    [5,2,1],   # node 4
    [1,2,4],   # node 5
]

In [3]:
def num_nodes(al):
    return len(al)

def num_edges(al):
    e = 0
    for n in al:
        e += len(n)
    return e//2

def degree(node):
    return len(node)

def neighbours(node):
    for n in node:
        print(n)

In [4]:
print(f"Number of nodes in adjacency list: {num_nodes(adjacency_list)}")
print(f"Number of edges in adjacency list: {num_edges(adjacency_list)}")
print(f"Degree of node 4 from adjacency list: {degree(adjacency_list[4])}")
print(f"Neighbours of node 4 from adjacency list...")
neighbours(adjacency_list[4])

Number of nodes in adjacency list: 6
Number of edges in adjacency list: 7
Degree of node 4 from adjacency list: 3
Neighbours of node 4 from adjacency list...
5
2
1


#### Transforming edge lists into adjacency lists
The above adjacency list could have been given as an edge list. Edge lists are less useful as it takes $O(E)$ time to find a node's neighbours from an edge list, compared with $O(degree(node))$ for an adjacency list. As such, it is generally a good idea to transform an edge list into an adjacency list.

Note that if searching for neighbours from an adjacency list becomes a bottleneck, we can transform the neighbour list into a set. Checking will then only require $O(1)$ time.

In [6]:
edges = [[0,1],[1,2],[1,4], [1,5], [2,4], [2,5], [4,5]]
V = 6

In [7]:
def build_adjacency_list(V, edges):
    graph = [[] for _ in range(V)]
    for n1, n2 in edges:
        graph[n1].append(n2)
        graph[n2].append(n1)
    return graph  

In [8]:
build_adjacency_list(6, edges)

[[1], [0, 2, 4, 5], [1, 4, 5], [], [1, 2, 5], [1, 2, 4]]